## 1. Install and Import libraries

In [3]:
# !pip install -U scikit-learn

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.4 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 643.6 kB/s eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.6 MB 3.3 MB/s eta 0:00:04
   - -------------------------------------- 0.3/10.6 MB 3.4 MB/s eta 0:00:04
   - -------------------------------------- 0.5/10.6 MB 4.3 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/10.6 MB 4.5 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/10.6 MB 4.7 MB/s eta 0:00:03
   ----- ---------------------------------- 1.5/10.6 MB 6.0 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/10.6 MB 6.3 MB/s eta 0:00:02
   ------ --------------

In [1]:
# Manipulating matrixes and DataFrames
import numpy as np
import pandas as pd

# Pre-build models
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale

C:\Users\dell\AppData\Local\Temp\ipykernel_17300\2427006838.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 2. Load and process data

In [2]:
# Load the hitters dataset
hitters = pd.read_csv('https://gist.githubusercontent.com/keeganhines/59974f1ebef97bbaa44fb19143f90bad/raw/d9bcf657f97201394a59fffd801c44347eb7e28d/Hitters.csv')

# Drop string columns
hitters = hitters.drop(columns=['League','NewLeague','Division', 'Unnamed: 0'])

# Dop missing values
hitters = hitters.dropna()

# Splitting into input and target variables
x = hitters.drop('Salary', axis=1)
y = hitters['Salary']

# Scaling data
x_scaled = scale(x)
y_scaled = scale(y)

## 3. Fit a Linear Regression Model

In [10]:
model = LinearRegression(fit_intercept=False)
model.fit(x_scaled, y_scaled)

LinearRegression(fit_intercept=False)

## 4. Estimate coefficients

### 4.1. Using the models attribute

We can determinate the OLS coefficients by using the build-in attribute `.coef`

In [11]:
print(model.coef_)

[-0.71936948  0.78300967  0.04199872 -0.11886574 -0.00131479  0.29612846
 -0.02754667 -0.89348966  0.10023847 -0.0424706   1.18205318  0.57447365
 -0.46472223  0.1827929   0.12349358 -0.0421584 ]


In [24]:
coefs_df = pd.DataFrame(data=model.coef_, columns=["Coefs - Attribute"], index=x.columns)
coefs_df

,Coefs - Attribute
AtBat,-0.719369
Hits,0.783010
HmRun,0.041999
Runs,-0.118866
RBI,-0.001315
Walks,0.296128
Years,-0.027547
CAtBat,-0.893490
CHits,0.100238
CHmRun,-0.042471


### 4.2. Manually

We can estimate the Linear regression coefficientes manually as well. For this we should take into account that Linear regression is expressed as:

$$ y = X\beta + \epsilon $$

where:
- $ X $ is the feature matrix,
- $ \beta $ is the vector of coefficients, and
- $ \epsilon $ is the vector of error terms.

We can find the optimal coefficients by minimizing the Residual Sum of Squares (RSS). RSS is defined as:

$$ \text{RSS} = (y - X\beta)'(y - X\beta) $$
$$ \text{RSS} = y'y - 2y'X\beta + \beta'X'X\beta $$

To minimize the RSS, we derive the loss function with respect to $ \beta $ and set it to zero. In other words, we are looking to find the critical point where the values of $ \beta $ lead to the lowest possible error. For this:

The derivative of RSS with respect to $ \beta $ is:

$$ \frac{\partial \text{RSS}}{\partial \beta} = -2X'y + 2X'X\beta $$

Setting the derivative to zero:

$$ -2X'y + 2X'X\beta = 0 $$

From this, we solve for $ \beta $, which takes the value:

$$ \hat{\beta} = (X'X)^{-1}X'y $$
$$ X'X\hat{\beta} = X'y $$

By solving this equation, we obtain the coefficient values that minimize the error, providing us with the best fit model.

In [23]:
Xtx = np.dot(x_scaled.T, x_scaled)
Xty = np.dot(x_scaled.T, y_scaled)
beta = np.linalg.solve(Xtx, Xty)
print(beta)

[-0.71936948  0.78300967  0.04199872 -0.11886574 -0.00131479  0.29612846
 -0.02754667 -0.89348966  0.10023847 -0.0424706   1.18205318  0.57447365
 -0.46472223  0.1827929   0.12349358 -0.0421584 ]


In [27]:
coefs_df['Coefs - Beta'] = beta
coefs_df

,Coefs - Attribute,Coefs - Beta
AtBat,-0.719369,-0.719369
Hits,0.783010,0.783010
HmRun,0.041999,0.041999
Runs,-0.118866,-0.118866
RBI,-0.001315,-0.001315
Walks,0.296128,0.296128
Years,-0.027547,-0.027547
CAtBat,-0.893490,-0.893490
CHits,0.100238,0.100238
CHmRun,-0.042471,-0.042471


This code performs matrix operations that correspond to the mathematical steps we've described:

- `np.dot(x_scaled.T, x_scaled)` calculates $ X'X $,
- `np.dot(x_scaled.T, y_scaled)` calculates $ X'y $,
- `np.linalg.solve()` solves the equation $ X'X\beta = X'y $ for $ \beta $,
- `print(beta)` then outputs the calculated coefficients.


## 5. Gradient Descent

The gradient descent algorithm is an optimization method used to find the minimum of a cost function. This process iteratively adjust the model's parameters to minimize the cost function. The update rule can be summarized by the following formula:

$$ w_{1} = w_{0} - \alpha \cdot (\nabla f(w_{0})) $$

- **Initial Weights (`w0`)**: Start with a randomly initialized set of coefficients.
- **Learning Rate (`alpha`)**: A predefined step size that determines how much we adjust the weights with respect to the gradient.
- **Gradient Calculation**: Compute the gradient of the cost function, which is the sum of squared residuals in linear regression.
- **Weights Update (`w1`)**: Modify the weights in the opposite direction of the gradient to minimize the cost function.
- **Convergence Check**: Continue iterating until the changes in weights are smaller than a defined threshold (`atol`), indicating that we've reached the minimum.
- **Final Weights Output**: Once convergence is achieved, output the optimized weights, which represent the best-fit coefficients for the linear regression model.


In [8]:
# Initialize weights randomly
w0 = np.random.uniform(size=x_scaled.shape[1])

# Set the learning rate
alpha = 0.0005

w1 = w0.copy()

# Set a loop that will continue untl break condition is met
while True:

    # Calculate predictions
    predictions = np.dot(x_scaled, w0)
    # Calculate errors
    errors = y_scaled - predictions
    # Calculate gradient (direction to adjust weights to minimize the loss function)
    gradient = -2 * np.dot(x_scaled.T, errors)
    # Update weights
    w1 = w0 - alpha * gradient

    # Check for convergence
    if np.allclose(w1, w0, atol=1e-12):
        break

    # Prepare for the next iteration
    w0 = w1.copy()

# Print final weights
print(w1)

[-0.71926295  0.78281164  0.04194828 -0.11874352 -0.00127453  0.29609038
 -0.02750359 -0.89432822  0.10180277 -0.04213224  1.18135805  0.57394527
 -0.46454222  0.18278814  0.12351221 -0.04217029]


In [28]:
coefs_df['Coefs - GD'] = w1
coefs_df

,Coefs - Attribute,Coefs - Beta,Coefs - GD
AtBat,-0.719369,-0.719369,-0.719263
Hits,0.783010,0.783010,0.782812
HmRun,0.041999,0.041999,0.041948
Runs,-0.118866,-0.118866,-0.118744
RBI,-0.001315,-0.001315,-0.001275
Walks,0.296128,0.296128,0.296090
Years,-0.027547,-0.027547,-0.027504
CAtBat,-0.893490,-0.893490,-0.894328
CHits,0.100238,0.100238,0.101803
CHmRun,-0.042471,-0.042471,-0.042132
